# H.E.S.S. with Gammapy

This tutorial explains how to analyse [H.E.S.S.](https://www.mpi-hd.mpg.de/hfm/HESS) data with Gammapy.

We will analyse four observation runs of the Crab nebula, which are part of the [H.E.S.S. first public test data release](https://www.mpi-hd.mpg.de/hfm/HESS/pages/dl3-dr1/). We will use the template background model from off runs produced in the [background_model.ipynb](background_model.ipynb) notebook with Gammapy.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from astropy.coordinates import SkyCoord
from gammapy.data import DataStore
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.cube import MapMaker, MapFit, PSFKernel
from gammapy.detect import TSMapEstimator

## Data access

To access the data, we use the `DataStore`, and we use the ``obs_table`` to select the Crab runs.

In [ ]:
data_store = DataStore.from_file(
    "$GAMMAPY_DATA/hess-dl3-dr1/hess-dl3-dr3-with-background.fits.gz"
)
mask = data_store.obs_table["TARGET_NAME"] == "Crab"
obs_table = data_store.obs_table[mask]
obs_list = data_store.obs_list(obs_table["OBS_ID"])

In [ ]:
# pos_crab = SkyCoord.from_name('Crab')
pos_crab = SkyCoord(83.633, 22.014, unit="deg")

## Maps

Let's make some 3D cubes, as well as 2D images.

In [ ]:
energy_axis = MapAxis.from_edges(
    np.logspace(0, 1., 10), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(83.633, 22.014),
    binsz=0.02,
    width=(5, 5),
    coordsys="CEL",
    proj="TAN",
    axes=[energy_axis],
)

In [ ]:
%%time
maker = MapMaker(geom, offset_max="2.5 deg")
maps = maker.run(obs_list)
images = maker.make_images()

In [ ]:
maps.keys()

In [ ]:
images["counts"].plot(stretch="sqrt");

## PSF

Compute the mean PSF for these observations at the Crab position.

In [ ]:
table_psf = obs_list.make_mean_psf(pos_crab)

In [ ]:
psf_kernel = PSFKernel.from_table_psf(table_psf, geom, max_radius="0.3 deg")
psf_kernel_array = psf_kernel.psf_kernel_map.sum_over_axes().data
# psf_kernel.psf_kernel_map.slice_by_idx({'energy': 0}).plot()
# plt.imshow(psf_kernel_array)

In [ ]:
%%time
images_ts = TSMapEstimator().run(images, psf_kernel_array)

In [ ]:
images_ts["sqrt_ts"].plot(vmax=20, add_cbar=True);